In [ ]:
import pandas as pd
import requests
import os
import shutil


#### **Notice**  
1. For data of volume, **Bitcon** data is incomplete, so we chose to eliminate the Bitcon in the volume section, and chose to plot the volume graph in the next place in the tvl order sort, i.e., the **Polygon**.  
2. For the chainname obtained through the tvl sort is not completely correct, in the api of the fee and volume there are some chains can not get the data, we are here to eliminate these data.  
3. We need data of 150 days to calculate the z-score to recognize abnormal values. We only plot data of 120 days.

In [24]:

# you can get the chain list straihgt from the API
url = 'https://api.llama.fi/v2/chains'
response = requests.get(url)
if response.status_code == 200: # data was fetched successfully
    data = response.json()
    df = pd.DataFrame(data)
else:
    raise Exception('Error fetching data from DeFiLlama API')

dex_all_chains_url = "https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"
dex_all_chains_response = requests.get(dex_all_chains_url)

dex_all_chains_data = dex_all_chains_response.json()
dex_all_chains = set(dex_all_chains_data.get('allChains', []))


fees_all_chains_url = "https://api.llama.fi/overview/fees?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyFees"
fees_all_chains_response = requests.get(fees_all_chains_url)

fees_all_chains_data = fees_all_chains_response.json()
fees_all_chains = set(fees_all_chains_data.get('allChains', []))

common_chains = dex_all_chains.intersection(fees_all_chains) # get the common chains between the two sets

df = df[['name', 'tvl']]
df = df[df['name'].isin(common_chains)]

# Sort the data to get the top 10 chains by TVL
df = df.sort_values(by='tvl', ascending=False).head(11)
print(df)

base_folder = './data'
fee_folder = os.path.join(base_folder, 'fee')
tvl_folder = os.path.join(base_folder, 'tvl')
volume_folder = os.path.join(base_folder, 'volume')

for folder in [fee_folder, tvl_folder, volume_folder]:
    if os.path.exists(folder):
        shutil.rmtree(folder)  # delete the folder and its contents
    os.makedirs(folder)

         name           tvl
14   Ethereum  7.347807e+10
91       Tron  1.368646e+10
46     Solana  9.185863e+09
6         BSC  5.922616e+09
8        Base  3.766777e+09
53    Bitcoin  3.576340e+09
15   Arbitrum  3.233247e+09
13  Avalanche  1.613274e+09
94        Sui  1.555979e+09
57      Aptos  1.266350e+09
16    Polygon  1.183367e+09


In [25]:
chain_order = df['name'].tolist()
chain_index = {chain: str(index + 1).zfill(2) for index, chain in enumerate(chain_order)}
# give chainname an index so we can plot in order


def fetch_historical_data(chain_name, metric):
    if metric == 'tvl':
        url = f'https://api.llama.fi/v2/historicalChainTvl/{chain_name}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)  # Directly create DataFrame from the list of dictionaries
            df['date'] = pd.to_datetime(df['date'], unit='s')
            csv_filename = os.path.join(tvl_folder, f"{chain_index[chain_name]}_{chain_name}_{metric}.csv")
            df = df[['date', metric]].tail(150)
            df.to_csv(csv_filename, index=False)
            print(f"Saved {metric} data for {chain_name} to {csv_filename}")
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'volume':
        url = (f'https://api.llama.fi/overview/dexs/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyVolume')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            df = df[['date', metric]].tail(150)
            csv_filename = os.path.join(volume_folder, f"{chain_index[chain_name]}_{chain_name}_{metric}.csv")
            df.to_csv(csv_filename, index=False)
            print(f"Saved {metric} data for {chain_name} to {csv_filename}")
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'fee':
        url = (f'https://api.llama.fi/overview/fees/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyFees')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            df = df[['date', metric]].tail(150)
            csv_filename = os.path.join(fee_folder, f"{chain_index[chain_name]}_{chain_name}_{metric}.csv")
            df.to_csv(csv_filename, index=False)
            print(f"Saved {metric} data for {chain_name} to {csv_filename}")
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
    else:
        raise ValueError('Invalid metric specified')

In [26]:
for chain_name in df['name'].head(10):
    for metric in ['tvl', 'fee']:
        fetch_historical_data(chain_name, metric)
        
for chain_name in df_volume['name']:
    if chain_name.lower() == 'bitcoin':
        print(f"Skipping {chain_name} for {metric}")
        continue # Skip Bitcoin as its volume data is incomplete
    metric = 'volume'
    fetch_historical_data(chain_name, metric)

Saved tvl data for Ethereum to ./data\tvl\01_Ethereum_tvl.csv
Saved fee data for Ethereum to ./data\fee\01_Ethereum_fee.csv
Saved tvl data for Tron to ./data\tvl\02_Tron_tvl.csv
Saved fee data for Tron to ./data\fee\02_Tron_fee.csv
Saved tvl data for Solana to ./data\tvl\03_Solana_tvl.csv
Saved fee data for Solana to ./data\fee\03_Solana_fee.csv
Saved tvl data for BSC to ./data\tvl\04_BSC_tvl.csv
Saved fee data for BSC to ./data\fee\04_BSC_fee.csv
Saved tvl data for Base to ./data\tvl\05_Base_tvl.csv
Saved fee data for Base to ./data\fee\05_Base_fee.csv
Saved tvl data for Bitcoin to ./data\tvl\06_Bitcoin_tvl.csv
Saved fee data for Bitcoin to ./data\fee\06_Bitcoin_fee.csv
Saved tvl data for Arbitrum to ./data\tvl\07_Arbitrum_tvl.csv
Saved fee data for Arbitrum to ./data\fee\07_Arbitrum_fee.csv
Saved tvl data for Avalanche to ./data\tvl\08_Avalanche_tvl.csv
Saved fee data for Avalanche to ./data\fee\08_Avalanche_fee.csv
Saved tvl data for Sui to ./data\tvl\09_Sui_tvl.csv
Saved fee data f

ModuleNotFoundError: No module named 'src'